In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Installing the required modules
!apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip install pyfluidsynth pretty_midi

!pip install magenta
!pip install note_seq
!pip install tensorflow

Selecting previously unselected package fluid-soundfont-gm.
(Reading database ... 160815 files and directories currently installed.)
Preparing to unpack .../fluid-soundfont-gm_3.1-5.1_all.deb ...
Unpacking fluid-soundfont-gm (3.1-5.1) ...
Selecting previously unselected package libfluidsynth1:amd64.
Preparing to unpack .../libfluidsynth1_1.1.9-1_amd64.deb ...
Unpacking libfluidsynth1:amd64 (1.1.9-1) ...
Setting up fluid-soundfont-gm (3.1-5.1) ...
Setting up libfluidsynth1:amd64 (1.1.9-1) ...
Processing triggers for libc-bin (2.27-3ubuntu1.2) ...
/sbin/ldconfig.real: /usr/local/lib/python3.7/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link

     |████████████████████████████████| 5.6MB 7.0MB/s 
     |████████████████████████████████| 51kB 7.0MB/s 
  Created wheel for pretty-midi: filename=pretty_midi-0.2.9-cp37-none-any.whl size=5591958 sha256=6e90568f958840e3b8b9bdebfc7f0f3e695a1c7eef26712a5bce150e3911621d
  Stored in directory: /root/.cache/pip/wheels/4c/a1/c6/b5697841

In [160]:
SESSION_PATH = '/content/drive/MyDrive/Master thesis/ableton sessions/D2 Project'
MISC_PATH = '/content/drive/MyDrive/Master thesis/track list/misc'
SESSION_NAME = 'D2'

In [161]:
import os
os.path.isdir(MISC_PATH)

True

In [162]:
csv_file = os.path.join(SESSION_PATH,'D2.csv')
os.path.isfile(csv_file)

True

In [163]:
import pandas as pd

df = pd.read_csv(csv_file)

df.head()

Unnamed: 0  Unnamed: 0.1  index  ... bad Syncopation      style
0         355           355    319  ...   0          15       rock
1         146           146    127  ...   0           6       rock
2         154           154    135  ...   0          21       rock
3          12            12     10  ...   0          13       rock
4         172           172    150  ...   0          37  afrocuban

[5 rows x 13 columns]

In [164]:
file_path_list = []
for file_path in df.file_name:
  file_path_list.append(file_path.replace('\\','/'))

file_path_list[0:5]

['validation/drummer2-session2-2_rock_130_beat_4-4_best_2bar_segment_4.mid',
 'train/drummer3-session1-9_rock_105_beat_4-4_best_2bar_segment_2.mid',
 'train/drummer3-session2-28_rock_92_beat_4-4_best_2bar_segment_0.mid',
 'test/drummer3-session1-11_rock_120_beat_4-4_best_2bar_segment_1.mid',
 'train/drummer5-session1-13_afrocuban-calypso_124_beat_4-4_best_2bar_segment_5.mid']

In [165]:
bpm_list = []
for bpm_ in df.bpm:
  bpm_list.append(float(bpm_))

bpm_list[0:5]

[130.0, 105.0, 92.0, 120.0, 124.0]

In [166]:
os.path.isfile(os.path.join(MISC_PATH,file_path_list[0]))

True

In [167]:
import copy

def change_tempo(note_sequence, new_tempo):
    new_sequence = copy.deepcopy(note_sequence)
    ratio = note_sequence.tempos[0].qpm / new_tempo
    for note in new_sequence.notes:
        note.start_time = note.start_time * ratio
        note.end_time = note.end_time * ratio
    new_sequence.tempos[0].qpm = new_tempo
    return new_sequence

In [168]:
import note_seq
from note_seq import sequences_lib, midi_io

file_name_list = []
dualised_name_list = []
track_num = 0
for file_path in file_path_list:
  midi_loc = os.path.join(MISC_PATH,file_path)
  tmp_note_seq = midi_io.midi_file_to_note_sequence(midi_loc)
  new_name = file_path.split('/')[-1][:-4]
    
  if track_num < 10:
    new_name = SESSION_NAME + '_track0' + str(track_num) + '_' + new_name
  else:
    new_name = SESSION_NAME + '_track' + str(track_num) + '_' + new_name
  
  current_bpm = bpm_list[track_num]
  track_num = track_num + 1

  file_name_list.append(new_name + '_original.mid')
  dualised_name_list.append(new_name + '_dualised.mid')

  new_save_path = os.path.join(SESSION_PATH, 'data')
  new_save_path = os.path.join(new_save_path, new_name + '_original.mid')
  midi_io.note_sequence_to_midi_file(tmp_note_seq, new_save_path)

  dualised_path = os.path.join(SESSION_PATH, 'raw_data')
  dualised_path = os.path.join(dualised_path, new_name + '_dualised.mid')
  tmp_note_seq = midi_io.midi_file_to_note_sequence(dualised_path)
  note_seq_correct_tempo = change_tempo(tmp_note_seq, current_bpm)

  new_save_path = os.path.join(SESSION_PATH, 'data')
  new_save_path = os.path.join(new_save_path, new_name + '_dualised.mid')
  midi_io.note_sequence_to_midi_file(note_seq_correct_tempo, new_save_path)

In [169]:
import numpy as np
session_list = [SESSION_NAME] * 68
track_list = np.arange(68)

In [170]:
track_list

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67])

In [171]:
import random
import glob
tmp_midi_file_list = glob.glob(os.path.join(SESSION_PATH, 'data/*_original.mid'))
#tmp_midi_file_list

In [172]:
tmp_midi_file = tmp_midi_file_list[random.randint(0,68)]
print(tmp_midi_file)
tmp_note_seq = midi_io.midi_file_to_note_sequence(tmp_midi_file)
print(tmp_note_seq.tempos)

/content/drive/MyDrive/Master thesis/ableton sessions/D2 Project/data/D2_track17_drummer9-session1-26_rock_90_beat_4-4_best_2bar_segment_0_original.mid
[qpm: 90.00009000009
]


In [173]:
df_new = df.drop("Unnamed: 0",axis=1)
df_new = df_new.drop("Unnamed: 0.1",axis=1)
df_new = df_new.drop("split",axis=1)
df_new = df_new.drop("beat_type",axis=1)
df_new = df_new.drop("time_signature",axis=1)
df_new = df_new.drop("good",axis=1)
df_new = df_new.drop("maybe",axis=1)
df_new = df_new.drop("bad",axis=1)
df_new['session'] = session_list
df_new['track'] = track_list
df_new['grooVAE_name'] = file_name_list
df_new['dualised_name'] = dualised_name_list
df_new

index  ...                                      dualised_name
0     319  ...  D2_track00_drummer2-session2-2_rock_130_beat_4...
1     127  ...  D2_track01_drummer3-session1-9_rock_105_beat_4...
2     135  ...  D2_track02_drummer3-session2-28_rock_92_beat_4...
3      10  ...  D2_track03_drummer3-session1-11_rock_120_beat_...
4     150  ...  D2_track04_drummer5-session1-13_afrocuban-caly...
..    ...  ...                                                ...
63    220  ...  D2_track63_drummer7-session3-108_rock_95_beat_...
64    168  ...  D2_track64_drummer5-session2-18_latin-brazilia...
65    224  ...  D2_track65_drummer7-session3-129_rock_93_beat_...
66      6  ...  D2_track66_drummer1-session1-28_latin-samba_11...
67    255  ...  D2_track67_drummer8-session1-2_funk_95_beat_4-...

[68 rows x 9 columns]

In [174]:
neworder = ['session','track','grooVAE_name','dualised_name','index','file_name',\
            'bpm','Syncopation','style']

df_new=df_new.reindex(columns=neworder)
df_new=df_new.rename(columns={'file_name': 'original_file_location'})
df_new

session  track  ... Syncopation      style
0       D2      0  ...          15       rock
1       D2      1  ...           6       rock
2       D2      2  ...          21       rock
3       D2      3  ...          13       rock
4       D2      4  ...          37  afrocuban
..     ...    ...  ...         ...        ...
63      D2     63  ...           8       rock
64      D2     64  ...          48      latin
65      D2     65  ...          12       rock
66      D2     66  ...          75      latin
67      D2     67  ...          18       funk

[68 rows x 9 columns]

In [175]:
df_new.to_csv(os.path.join(SESSION_PATH, SESSION_NAME + '.csv'),index=False)